In [1]:
# modulos
import sympy as sp
import numpy as np

# simbolos generales
s,R1,R2,R3,R4,R5,R6,C1,C2,C3,C4,Vi,Vo,Vx,V,k,Vy=sp.symbols("s,R1,R2,R3,R4,R5,R6,C1,C2,C3,C4,Vi,Vo,Vx,V,k,Vy")

g1=1/R1
g2=1/R2
g3=1/R3
g4=1/R4
X2=1/(s*C2)

# print rapido de ecuaciones
def pp(eqs):
    print("\n")
    [sp.pprint(eq,wrap_line=False) for eq in eqs]
    print("\n")

# print rapido de separadores
def pr(txt=None):
    # si no hay texto, imprimir linea de fin
    if txt==None:
        print(f"{'*'*100}\n\n")
    
    # sino, imprimir un cartel de inicio
    else:
        tittle=f"|{txt}|".center(2*len(txt))
        
        print("-"*len(tittle))
        print(tittle)
        print("-"*len(tittle))


# Negative feedback
!["negFeed"](./img/negFeedBP.jpg)

In [2]:
# ecuacion de circuito
pr("Ecuaciones de nodos.")

# considerando el nodo X (R1,C1,C2)
nodo0=sp.Eq(Vx*(g1+s*C1+s*C2)-Vo*s*C2-g1*Vi,0)
# teniendo en cuenta que la corriente por C1 debe ser igual a la corriente por R2
nodo1=sp.Eq(Vo*g2,-Vx*s*C1)

pp([nodo0,nodo1])

# sp.print_latex(nodo0)
# sp.print_latex(nodo1)

pr()

# -----------------------------------------------------------------------------------
# buscar funcion de transferencia
pr("Funcion de transferencia.")

# resolver para vo y vi, y dividir
sol0=sp.solve((nodo0,nodo1),(Vo,Vi))

T=sp.collect(sp.simplify(sol0[Vo]/sol0[Vi]),s)
pp([sp.Eq(Vo/Vi,T)])

pr()

# -----------------------------------------------------------------------------------
# normalizar 
pr("Normalizacion.")

# recuperar num den
n,d=sp.fraction(T)

# normalizar coeficiente principal del denominador
n=sp.simplify(n/d.coeff(s,2))
d=sp.simplify(d/d.coeff(s,2))

T=sp.collect(n/d,s)
pp([sp.Eq(sp.symbols("T"),T)])

# sp.print_latex(T)

pr()

----------------------------------------
         |Ecuaciones de nodos.|         
----------------------------------------


              ⎛              1 ⎞   Vi    
-C₂⋅Vo⋅s + Vx⋅⎜C₁⋅s + C₂⋅s + ──⎟ - ── = 0
              ⎝              R₁⎠   R₁    
Vo           
── = -C₁⋅Vx⋅s
R₂           


****************************************************************************************************


--------------------------------------------------
           |Funcion de transferencia.|            
--------------------------------------------------


Vo                 -C₁⋅R₂⋅s                
── = ──────────────────────────────────────
Vi                2                        
     C₁⋅C₂⋅R₁⋅R₂⋅s  + s⋅(C₁⋅R₁ + C₂⋅R₁) + 1


****************************************************************************************************


----------------------------
      |Normalizacion.|      
----------------------------


                        -s                      
T = ────────────────────────

## Diseño

In [3]:
# requisotos
k2=3126.49 
a2=1895.58 
b2=23500096

# funciones de transferencia
pr("Transferencias")

T2=T

pp([sp.Eq(T2,-k2*s/(s**2+a2*s+b2))])
# sp.print_latex(sp.Eq(T1,-k1*s/(s**2+a1*s+b1)))
pr()

# igualacion de coef
print("Igualaciones")

# extraer y normalizar num y den
n2,d2=sp.fraction(T2)
cp=d2.expand().coeff(s,2)

ig1=sp.Eq((d2/cp).coeff(s,1),a2)
ig2=sp.Eq((d2/cp).coeff(s,0),b2)
pp([ig1,ig2])
pr()

# valuando componentes
pr("Valuando componentes")

c1=1
c2=1

ig1=ig1.subs({C1:c1,C2:c2})
ig2=ig2.subs({C1:c1,C2:c2})
pp([ig1,ig2])
# sp.print_latex(ig1)
# sp.print_latex(ig2)
pr()


# calculo de componentes
print("Valores")

[(r1,r2)]=sp.solve((ig1,ig2),(R1,R2))

# componentes
escala1=10E6
escala2=10E6

pp([sp.Eq(C1,c1/escala1),sp.Eq(C2,c2/escala1),sp.Eq(R1,r1*escala1),sp.Eq(R2,r2*escala2)])

# sp.print_latex(sp.Eq(C1,c1/escala1))
# sp.print_latex(sp.Eq(C2,c2/escala1))
# sp.print_latex(sp.Eq(R,r1*escala1))
# sp.print_latex(sp.Eq(k,k1))
pr()

pr("valores de implementacion")
c1=100E-9
c2=100E-9
r1=410
r2=10560

----------------------------
      |Transferencias|      
----------------------------


                    -s                                -3126.49⋅s        
──────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1  ⎞        1     ⎞    2                       
C₂⋅R₁⋅⎜s  + s⋅⎜───── + ─────⎟ + ───────────⎟   s  + 1895.58⋅s + 23500096
      ⎝       ⎝C₂⋅R₂   C₁⋅R₂⎠   C₁⋅C₂⋅R₁⋅R₂⎠                            


****************************************************************************************************


Igualaciones


  1       1            
───── + ───── = 1895.58
C₂⋅R₂   C₁⋅R₂          
     1                
─────────── = 23500096
C₁⋅C₂⋅R₁⋅R₂           


****************************************************************************************************


----------------------------------------
         |Valuando componentes|         
----------------------------------------


2           
── = 1895.58
R₂          
  1             

## Sensitividades

In [4]:
pr("Simplificaciones")

# simbolos auxiliares
w1,w12,w2,w21=sp.symbols("w1,w12,w2,w21")

# simplificar ft
T0=T.subs({1/(R1*C1):w1,1/(R2*C2):w2}).subs({1/(R1*C2):w12,
                                             1/(R2*C1):w21})
pp([sp.Eq(T,T0)])

pr()

# --------------------------------------------------------------------------------------
pr("Parametros de analisis")
n0,d0=sp.fraction(T0)

wp2=sp.factor(d0.coeff(s,0))
bwp=sp.factor(d0.coeff(s,1))

pp([sp.Eq(sp.sympify("wp**2"),wp2),sp.Eq(sp.sympify("bwp"),bwp)])

pr()

--------------------------------
       |Simplificaciones|       
--------------------------------


                    -s                                  -s⋅w₁₂          
──────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1  ⎞        1     ⎞    2                       
C₂⋅R₁⋅⎜s  + s⋅⎜───── + ─────⎟ + ───────────⎟   s  + s⋅(w₂ + w₂₁) + w₁⋅w₂
      ⎝       ⎝C₂⋅R₂   C₁⋅R₂⎠   C₁⋅C₂⋅R₁⋅R₂⎠                            


****************************************************************************************************


--------------------------------------------
          |Parametros de analisis|          
--------------------------------------------


  2        
wp  = w₁⋅w₂
bwp = w₂ + w₂₁


****************************************************************************************************




### Sensitividades Wp:

In [5]:
# -----------------------------------------------------------------------------------------
pr("Sensitividad para wp.")
pp([sp.Eq(sp.sympify("wp**2"),wp2)])
sp.print_latex(sp.Eq(sp.sympify("wp**2"),wp2))

""" wp2 == w₁⋅w₂ """

# formato: pssx == [pssR1, pssR2,... ,pssRn, pssC1, pssC2,... , pssCn]
w1ssx=np.array([-1,0,-1,0])
w12ssx=np.array([-1,0,0,-1])
w2ssx=np.array([0,-1,0,-1])
w21ssx=np.array([0,-1,-1,0])

# propiedad: (p1*p2)Sx == p1Sx + p2Sx
wp2ssx=w1ssx+w2ssx

# propiedad: (p**n)Sx == n*(pSx)
wpssx=wp2ssx/2

# generar un diccionario
keys=[R1,R2,C1,C2]

wpStv={k:sp.simplify(v) for k,v in zip(keys,wpssx)}
pp([wpStv])
sp.print_latex(wpStv)

# comprobar los calculos con identidad para wp
# identidad: sum(wpssC) == sum(wpssR) == -1
wpSumR=sp.simplify(wpssx[0]+wpssx[1])
wpSumC=sp.simplify(wpssx[2]+wpssx[3])
pp([sp.Eq(sp.Sum(sp.symbols("wpSsR_i"), (sp.symbols("i"), 1, 2)),wpSumR)])
pp([sp.Eq(sp.Sum(sp.symbols("wpSsC_i"), (sp.symbols("i"), 1, 2)),wpSumC)])
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("wpSsR_i"), (sp.symbols("i"), 1, 2)),wpSumR))
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("wpSsC_i"), (sp.symbols("i"), 1, 2)),wpSumC))
pr()

pr("Sensibilidades numericas")

# obtener valores numericos
NUMwpStv={k:wpStv[k].subs({R1:r1,R2:r2,C1:c1,C2:c2}) for k in keys}

pp([NUMwpStv])

------------------------------------------
         |Sensitividad para wp.|          
------------------------------------------


  2        
wp  = w₁⋅w₂


\wp^{2} = w_{1} w_{2}


{C₁: -0.5, C₂: -0.5, R₁: -0.5, R₂: -0.5}


\left\{ C_{1} : -0.5, \  C_{2} : -0.5, \  R_{1} : -0.5, \  R_{2} : -0.5\right\}


  2                
 ___               
 ╲                 
  ╲                
  ╱   wpSsRᵢ = -1.0
 ╱                 
 ‾‾‾               
i = 1              




  2                
 ___               
 ╲                 
  ╲                
  ╱   wpSsCᵢ = -1.0
 ╱                 
 ‾‾‾               
i = 1              


\sum_{i=1}^{2} wpSsR_{i} = -1.0
\sum_{i=1}^{2} wpSsC_{i} = -1.0
****************************************************************************************************


------------------------------------------------
           |Sensibilidades numericas|           
------------------------------------------------


{C₁: -0.5, C₂: -0.5, R₁: -0.5, R₂: -0.5}




### Sensitividades bwp:

In [6]:
# -----------------------------------------------------------------------------------------
pr("Sensitividad para bwp.")
pp([sp.Eq(sp.sympify("bwp"),bwp)])
sp.print_latex(sp.Eq(sp.sympify("bwp"),bwp))

""" bwp = w₂ + w₂₁ """ 
# sensitividades basicas
# w1ssx=np.array([-1,0,-1,0])
# w12ssx=np.array([-1,0,0,-1])
# w2ssx=np.array([0,-1,0,-1])
# w21ssx=np.array([0,-1,-1,0])

# propiedad: (p1+p2+ ...+pn)Sx == (p1*p1Sx + p2*p2Sx + ...+ pn*pnSx) / (p1+p2+ ...+pn)
bwpssx=(w2*w2ssx + w21*w21ssx)/(bwp)

# generar un diccionario
keys=[R1,R2,C1,C2]

bwpStv={k:sp.simplify(v.subs({w1:1/(R1*C1),
                              w2:1/(R2*C2),
                              w12:1/(R1*C2),
                              w21:1/(R2*C1)})) for k,v in zip(keys,bwpssx)}

pp([bwpStv])
sp.print_latex(bwpStv)

# comprobar los calculos con identidad para Qp
# identidad: sum(QpssC) == sum(QpssR) == 0

""" Aca se hace de forma indirecta porque tenemos las bwpSx pero no las QpSx. Pero aplicando propiedades se hace sencillo: """
# bwp == wp/Qp
# propiedad: (p1/p2)Sx == p1Sx - p2Sx
# bwpSx == (wp/Qp)Sx == wpSx - QpSx ==> QpSx == wpSx - bwpSx (calculadas)

QpSumR=sp.simplify((wpssx[0]-bwpssx[0])+(wpssx[1]-bwpssx[1]))
QpSumC=sp.simplify((wpssx[2]-bwpssx[2])+(wpssx[3]-bwpssx[3]))
pp([sp.Eq(sp.Sum(sp.symbols("QpSsR_i"), (sp.symbols("i"), 1, 2)),QpSumR)])
pp([sp.Eq(sp.Sum(sp.symbols("QpSsC_i"), (sp.symbols("i"), 1, 2)),QpSumC)])
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("QpSsR_i"), (sp.symbols("i"), 1, 2)),QpSumR))
sp.print_latex(sp.Eq(sp.Sum(sp.symbols("QpSsC_i"), (sp.symbols("i"), 1, 2)),QpSumC))
pr()

pr("Sensibilidades numericas")

# obtener valores numericos
NUMbwpStv={k:bwpStv[k].subs({R1:r1,R2:r2,C1:c1,C2:c2}) for k in keys}

pp([NUMbwpStv])

--------------------------------------------
          |Sensitividad para bwp.|          
--------------------------------------------


bwp = w₂ + w₂₁


bwp = w_{2} + w_{21}


⎧      -C₂          -C₁                 ⎫
⎨C₁: ───────, C₂: ───────, R₁: 0, R₂: -1⎬
⎩    C₁ + C₂      C₁ + C₂               ⎭


\left\{ C_{1} : - \frac{C_{2}}{C_{1} + C_{2}}, \  C_{2} : - \frac{C_{1}}{C_{1} + C_{2}}, \  R_{1} : 0, \  R_{2} : -1\right\}


  2             
 ___            
 ╲              
  ╲             
  ╱   QpSsRᵢ = 0
 ╱              
 ‾‾‾            
i = 1           




  2             
 ___            
 ╲              
  ╲             
  ╱   QpSsCᵢ = 0
 ╱              
 ‾‾‾            
i = 1           


\sum_{i=1}^{2} QpSsR_{i} = 0
\sum_{i=1}^{2} QpSsC_{i} = 0
****************************************************************************************************


------------------------------------------------
           |Sensibilidades numericas|           
-------------------------------

# Desviaciones

In [7]:
""" Dp/p == pSsR1*DR1/R1 + pSsR2*DR2/R2 + pSsC1*DC1/C1 + pSsC2*DC2/C2 == (VvR)*SUM[pSsRi] + (VvC)*SUM[pSsCi] """

# tolerancia de los componentes (10%)
VvR=0.1
VvC=0.1

pr("Cambio por unidad para Wp")

Vvwp=VvR*(NUMwpStv[R1]+NUMwpStv[R2]) + VvC*(NUMwpStv[C1]+NUMwpStv[C2])

print(f"{Vvwp}\n")
pr()

pr("Cambio por unidad para BWp")

Vvbwp=VvR*(NUMbwpStv[R1]+NUMbwpStv[R2]) + VvC*(NUMbwpStv[C1]+NUMbwpStv[C2])

print(f"{Vvbwp}\n")

--------------------------------------------------
           |Cambio por unidad para Wp|            
--------------------------------------------------
-0.200000000000000

****************************************************************************************************


----------------------------------------------------
            |Cambio por unidad para BWp|            
----------------------------------------------------
-0.200000000000000

